In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install captum


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import LayerIntegratedGradients
import pickle

In [ ]:
class InceptionModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionModel, self).__init__()

        # Stack 1
        self.stack1_branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU(inplace=True)
        )
        self.stack1_branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.stack1_branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )
        self.stack1_branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Stack 2
        self.stack2_branch1x1 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.ReLU(inplace=True)
        )
        self.stack2_branch3x3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.stack2_branch5x5 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )
        self.stack2_branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(128, 64, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Stack 3
        self.stack3_branch1x1 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1),
            nn.ReLU(inplace=True)
        )
        self.stack3_branch3x3 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.stack3_branch5x5 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )
        self.stack3_branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(256, 128, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Fully connected layer for classification
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # Stack 1
        stack1_branch1x1 = self.stack1_branch1x1(x)
        stack1_branch3x3 = self.stack1_branch3x3(x)
        stack1_branch5x5 = self.stack1_branch5x5(x)
        stack1_branch_pool = self.stack1_branch_pool(x)
        stack1_concatenated = torch.cat([stack1_branch1x1, stack1_branch3x3, stack1_branch5x5, stack1_branch_pool], dim=1)

        # Stack 2
        stack2_branch1x1 = self.stack2_branch1x1(stack1_concatenated)
        stack2_branch3x3 = self.stack2_branch3x3(stack1_concatenated)
        stack2_branch5x5 = self.stack2_branch5x5(stack1_concatenated)
        stack2_branch_pool = self.stack2_branch_pool(stack1_concatenated)
        stack2_concatenated = torch.cat([stack2_branch1x1, stack2_branch3x3, stack2_branch5x5, stack2_branch_pool], dim=1)

        # Stack 3
        stack3_branch1x1 = self.stack3_branch1x1(stack2_concatenated)
        stack3_branch3x3 = self.stack3_branch3x3(stack2_concatenated)
        stack3_branch5x5 = self.stack3_branch5x5(stack2_concatenated)
        stack3_branch_pool = self.stack3_branch_pool(stack2_concatenated)
        stack3_concatenated = torch.cat([stack3_branch1x1, stack3_branch3x3, stack3_branch5x5, stack3_branch_pool], dim=1)

        # Global Average Pooling
        pooled = self.global_avg_pool(stack3_concatenated)

        # Flatten the output for the fully connected layer
        flattened = pooled.view(pooled.size(0), -1)

        # Fully connected layer for classification
        output = self.fc(flattened)

        return output

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_subset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_subset = datasets.MNIST('.', train=False, download=True, transform=transform)

# Define the size of the random subsets
train_indices = torch.randperm(len(train_subset))[:30000]
test_indices = torch.randperm(len(test_subset))[:1000]

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=16, sampler=torch.utils.data.SubsetRandomSampler(test_indices))


# Create and train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create an instance of the Inception-like 3-stack model
in_channels = 1  # Input channels (e.g., for RGB images)
num_classes = 10  # Number of classes in your classification task
model = InceptionModel(in_channels, num_classes).to(device)

# Print the model architecture
print(model)
# Create and train the model on the GPU
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


100%|██████████| 9912422/9912422 [00:00<00:00, 73682616.77it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 34879266.87it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 19913249.30it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5679942.98it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



InceptionModel(
  (stack1_branch1x1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (stack1_branch3x3): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
  )
  (stack1_branch5x5): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): ReLU(inplace=True)
  )
  (stack1_branch_pool): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (stack2_branch1x1): Sequential(
    (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (stack2_branch3x3): Sequential(
    (0): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(64, 64, kernel_s

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

In [ ]:
# Train for 5 epochs as an example
'''for epoch in range(5):
    train(epoch)
    test(epoch)
file_path = '/content/drive/My Drive/InterpretingNN/code/model saved/sample_odel.pth'

# Save the model to a file
torch.save(model.state_dict(), file_path)'''

"for epoch in range(5):\n    train(epoch)\n    test(epoch)\nfile_path = '/content/drive/My Drive/InterpretingNN/code/model saved/sample_odel.pth'\n\n# Save the model to a file\ntorch.save(model.state_dict(), file_path)"

In [ ]:
# Load the model's state dictionary from the file
model = InceptionModel(in_channels, num_classes).to(device)
file_path = '/content/drive/My Drive/InterpretingNN/code/model saved/sample_odel.pth'
model.load_state_dict(torch.load(file_path))

# Put the model in evaluation mode (if needed)
model.eval()
print("loaded")

loaded


In [ ]:
def calculate_integrated_gradients_one_layer(model, layer, input_data, target_class):
    lig = LayerIntegratedGradients(model, layer)
    attribution = lig.attribute(input_data, target=target_class)

    return attribution

In [ ]:
def get_all_attributions_one_input(model, input_data, target_class):
    all_attributions = {}
    for name, layer in model.named_children():
      print(name)
      all_attributions[layer] = []
      layer_attributions=calculate_integrated_gradients_one_layer(model, layer, input_data, target_class)
      all_attributions[layer]=layer_attributions
    return all_attributions

In [ ]:
sample_input, target_class = next(iter(test_loader))
print(target_class)
sample_input.requires_grad_()
device = torch.device("cpu")
sample_input=sample_input.to(device)
model.to(device)
mode=model
num_class=10
# Calculate and print importance scores for each layer and neuron
attributions=get_all_attributions_one_input(model, sample_input, target_class)
file_path = 'attributions.pkl'

# Save the dictionary to a Pickle file
with open(file_path, 'wb') as file:
    pickle.dump(attributions, file)

# Load the dictionary from the Pickle file
'''with open(file_path, 'rb') as file:
    loaded_dict = pickle.load(file)

print("Loaded Dictionary (Pickle):", loaded_dict)'''

tensor([9, 8, 1, 1, 9, 8, 3, 3, 5, 4, 2, 6, 2, 6, 8, 2, 8, 0, 4, 9, 1, 8, 9, 1,
        7, 2, 0, 7, 0, 0, 5, 5, 2, 1, 8, 3, 8, 3, 3, 3, 0, 0, 8, 7, 1, 3, 9, 5,
        1, 8, 0, 3, 1, 6, 4, 4, 1, 2, 6, 4, 3, 5, 7, 8])
stack1_branch1x1
stack1_branch3x3
stack1_branch5x5
stack1_branch_pool
stack2_branch1x1
stack2_branch3x3
stack2_branch5x5
stack2_branch_pool
stack3_branch1x1
stack3_branch3x3
stack3_branch5x5
stack3_branch_pool
global_avg_pool
fc


'with open(file_path, \'rb\') as file:\n    loaded_dict = pickle.load(file)\n\nprint("Loaded Dictionary (Pickle):", loaded_dict)'